In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channel=3, num_classes=200):
        super(CNN, self).__init__()
        """
        Raj, S., Garyali, S., Kumar, S., & Shidnal, S. (2020). Image based bird species identification using convolutional neural network. International Journal of Engineering Research & Technology (IJERT), 9(6), 346.
        """
        self.conv1 = nn.Conv2d(in_channel, out_channels=32, kernel_size=(3, 3), padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=(3, 3))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=124, kernel_size=(3, 3), padding=1)
        self.fc1 = nn.Linear(124 * 8 * 8, 1024) 
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.50)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.dropout1(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        return x 

In [ ]:
my_transforms = transforms.Compose([
    transforms.Lambda(lambda img: Image.fromarray(img) if isinstance(img, np.ndarray) else img),  
    transforms.Lambda(lambda img: img.convert('RGB')),  
    transforms.RandomApply([transforms.RandomHorizontalFlip()], p=0.2),
    transforms.RandomApply([transforms.RandomRotation(10)], p=0.15),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)], p=0.2),
    transforms.RandomPerspective(distortion_scale=0.1, p=0.05),
    transforms.RandomApply([transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=5)], p=0.05),
    transforms.RandomApply([transforms.Grayscale(num_output_channels=3)], p=0.05),
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])